# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

# Import panel.interact
from panel.interact import interact

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAPBOX")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [ ]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path, index_col='neighbourhood')

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [ ]:
# Getting the data from the top 10 expensive neighbourhoods
df_top_10_expensive_neighbourhoods = (
    # select neighbourhood and average house value columns
    to_data[['neighbourhood', 'average_house_value']]
        # group by neighbourhood
        .groupby(['neighbourhood'])
        # calculate the mean
        .mean()
        # sort the values with the most expensive first
        .sort_values(by=['average_house_value'], ascending=False)
        # select only the top 10 (most expensive)
        .iloc[:10]
)

# Dwelling Type Columns
dwelling_types = [
    'single_detached_house',
    'apartment_five_storeys_plus',
    'movable_dwelling',
    'semi_detached_house',
    'row_house',
    'duplex',
    'apartment_five_storeys_less',
    'other_house',
]

# Calculate the sum number of dwelling types units per year
df_dwelling_types_sum = to_data[dwelling_types].groupby([to_data.index]).sum()

# Calculate the average monthly shelter costs for owned and rented dwellings
shelter_cols = ['shelter_costs_owned', 'shelter_costs_rented']
df_shelter_costs = to_data[shelter_cols].groupby([to_data.index]).mean()

### Panel Visualization Functions

In [ ]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
    # Calculate the mean values for each neighborhood
    df_mean_value_neighbourhoods = to_data.groupby(['neighbourhood']).mean()
    # Add coordinates
    df_neighbourhood_locations_mean = df_neighbourhood_locations.join(df_mean_value_neighbourhoods).reset_index()
    map_plot = px.scatter_mapbox(
        df_neighbourhood_locations_mean,
        title='Average House Values in Toronto',
        lat='lat',
        lon='lon',
        size='average_house_value',
        color='average_house_value',
        zoom=9,
        mapbox_style='open-street-map',
        hover_name='neighbourhood',
        hover_data=df_neighbourhood_locations_mean.columns,
    )

    return map_plot

def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    return data.hvplot.bar(
        title=title,
        xlabel=xlabel,
        ylabel=ylabel,
        color=color,
        rot=90,
        yformatter='%.0f',
        height=500,
    )

def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    return data.hvplot.line(
        title=title,
        xlabel=xlabel,
        ylabel=ylabel,
        color=color,
        yformatter='%.0f'
    )

def average_house_value():
    """Average house values per year."""
    # Calculate the average house value per year
    return create_line_chart(
        to_data['average_house_value'].groupby([to_data.index]).mean(),
        title='Average House Value in Toronto',
        xlabel='Year',
        ylabel='Avg. House Value',
        color='blue',
    )

def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    df_house_value_neighbourhood = to_data[['neighbourhood', 'average_house_value']]
    return interact(
        lambda neighbourhood: create_line_chart(
            df_house_value_neighbourhood[df_house_value_neighbourhood['neighbourhood'] == neighbourhood],
            title=f'neighbourhood: {neighbourhood}',
            xlabel='Year',
            ylabel='Avg. House Value',
            color='blue',
        ),
        neighbourhood=df_house_value_neighbourhood['neighbourhood'].unique()
    )

def number_dwelling_types():
    """Number of dwelling types per year"""
    df_dwelling_types = to_data[dwelling_types + ['neighbourhood']].groupby(['neighbourhood', to_data.index]).sum()
    # Use hvplot to create an interactive bar chart of the number of dwelling types per neighbourhood
    # The plot should have a dropdown selector for the neighbourhood
    return interact(
        lambda neighbourhood: df_dwelling_types.loc[neighbourhood].hvplot.bar(
            title=f'neighbourhood: {neighbourhood}',
            xlabel='Year',
            ylabel='Dwelling Type Units',
            rot=90,
            yformatter='%.0f',
            height=500,
        ),
        neighbourhood=df_dwelling_types.index.levels[0]
    )

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    # Calculate the average house value per year
    return create_line_chart(
        to_data['average_house_value'].groupby([to_data.index]).mean(),
        title='Average House Value in Toronto',
        xlabel='Year',
        ylabel='Avg. House Value',
        color='blue',
    )

def average_value_by_neighbourhood_and_year():
    """Average House Values in Toronto per Neighbourhood"""
    return px.bar(
        # Select neighbourhood and average_house_value columns
        to_data[['neighbourhood', 'average_house_value']],
        # Set plot title
        title='Average House Values in Toronto per Neighbourhood',
        # column of x values
        x='neighbourhood',
        # column of y values
        y='average_house_value',
        # Set color scale according to average house value
        color='average_house_value',
        # group data by year
        facet_row=to_data.index,
        # Display average_house_value as 'Avg. House Value'
        labels={
            'average_house_value': 'Avg. House Value',
        },
        # Set height of the plot
        height=1000,
    )

def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    df_top_10_expensive_neighbourhoods = (
        # select neighbourhood and average house value columns
        to_data[['neighbourhood', 'average_house_value']]
            # group by neighbourhood
            .groupby(['neighbourhood'])
            # calculate the mean
            .mean()
            # sort the values with the most expensive first
            .sort_values(by=['average_house_value'], ascending=False)
            # select only the top 10 (most expensive)
            .iloc[:10]
    )
    return df_top_10_expensive_neighbourhoods.hvplot.bar(
        title='Top 10 Expensive Neighbourhoods in Toronto',
        xlabel='Neighbourhood',
        ylabel='Avg. House Value',
        height=500,
        rot=90,
        yformatter='%.0f',
    )

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    # Fetch the data from all expensive neighbourhoods per year.
    # Select neighbourhood and average house value columns
    df_top_10_expensive_neighbourhoods_year = to_data[['neighbourhood', 'average_house_value']]
    # Select the top 10 most expensive neighbourhoods
    df_top_10_expensive_neighbourhoods_year = df_top_10_expensive_neighbourhoods_year[to_data['neighbourhood'].isin(df_top_10_expensive_neighbourhoods.index)]
    # Reset index and make the year a column
    df_top_10_expensive_neighbourhoods_year.reset_index(inplace=True)
    fig = px.sunburst(
        df_top_10_expensive_neighbourhoods_year,
        title='Costs Analysis of Most Expensive Neighbourhoods in Toronto per Year',
        path=['year', 'neighbourhood'],
        values='average_house_value',
        color='average_house_value',
        labels={
            'average_house_value': 'Avg. House Value',
        },
    )

    return fig

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
# YOUR CODE HERE!

# Define a welcome text
# YOUR CODE HERE!

# Create a tab layout for the dashboard
# YOUR CODE HERE!


# Create the main dashboard
# YOUR CODE HERE!

## Serve the Panel Dashboard

In [ ]:
dashboard.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()